In [20]:
import imageio
import os
import json

dds_folder = r"0ad-textures-1"
output_image_folder = "PNG"
output_metadata_folder = "Metadata"

os.makedirs(output_image_folder, exist_ok=True)
os.makedirs(output_metadata_folder, exist_ok=True)

def extract_dds_metadata(dds_path):
    metadata = {}
    with open(dds_path, "rb") as f:
        header = f.read(128)  # Standard DDS header
        if header[0:4] != b'DDS ':
            raise ValueError("Invalid DDS magic number")

        # Extract FourCC (compression format) from the DDS_PIXELFORMAT (at offset 84)
        fourcc = header[84:88]
        fourcc_str = fourcc.decode("ascii", errors="ignore")
        metadata["magic"] = "DDS "
        metadata["header_hex"] = header.hex()
        metadata["fourcc"] = fourcc_str

        # Check for DX10 extended header
        if fourcc_str == "DX10":
            dx10_header = f.read(20)
            metadata["dx10_header_hex"] = dx10_header.hex()
            metadata["has_dx10_header"] = True
        else:
            metadata["has_dx10_header"] = False

    return metadata

for filename in os.listdir(dds_folder):
    if filename.lower().endswith(".dds"):
        dds_path = os.path.join(dds_folder, filename)

        # Load image
        try:
            image = imageio.imread(dds_path, format='dds')
        except Exception as e:
            print(f"[!] Failed to load DDS image: {filename} → {e}")
            continue

        # Save as PNG
        png_filename = os.path.splitext(filename)[0] + ".png"
        png_path = os.path.join(output_image_folder, png_filename)
        imageio.imwrite(png_path, image, format='png')

        # Extract and save metadata
        try:
            metadata = extract_dds_metadata(dds_path)
            meta_filename = os.path.splitext(filename)[0] + ".json"
            meta_path = os.path.join(output_metadata_folder, meta_filename)
            with open(meta_path, "w") as f:
                json.dump(metadata, f, indent=2)
            print(f"[✓] {filename} → {png_filename} + metadata ({'DX10' if metadata['has_dx10_header'] else 'std'})")
        except Exception as e:
            print(f"[!] Failed to extract metadata: {filename} → {e}")


C:\Users\hp\AppData\Local\Temp\ipykernel_22456\1244544625.py:42: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imageio.imread(dds_path, format='dds')


[✓] alpine_cliff.dds → alpine_cliff.png + metadata (std)
[✓] alpine_cliff_a.dds → alpine_cliff_a.png + metadata (std)
[✓] alpine_cliff_b.dds → alpine_cliff_b.png + metadata (std)
[✓] alpine_cliff_c.dds → alpine_cliff_c.png + metadata (std)
[✓] alpine_cliff_snow.dds → alpine_cliff_snow.png + metadata (std)
[✓] alpine_dirt.dds → alpine_dirt.png + metadata (std)
[✓] alpine_dirt_grass_50.dds → alpine_dirt_grass_50.png + metadata (std)
[✓] alpine_dirt_snow.dds → alpine_dirt_snow.png + metadata (std)
[✓] alpine_forrestfloor.dds → alpine_forrestfloor.png + metadata (std)
[✓] alpine_forrestfloor_snow.dds → alpine_forrestfloor_snow.png + metadata (std)
[✓] alpine_grass_rocky.dds → alpine_grass_rocky.png + metadata (std)
[✓] alpine_grass_snow_50.dds → alpine_grass_snow_50.png + metadata (std)
[✓] alpine_mountainside.dds → alpine_mountainside.png + metadata (std)
[✓] alpine_shore_rocks.dds → alpine_shore_rocks.png + metadata (std)
[✓] alpine_shore_rocks_grass_50.dds → alpine_shore_rocks_grass_50.

In [2]:
import vpk
import os

vpk_path = 'VPK/tf2_misc_dir.vpk'
output_dir = 'ExtractedVPK/'

with vpk.open(vpk_path) as archive:
    for file_path in archive:
        if file_path.endswith('.vtf'):
            print("Extracting:", file_path)
            file_data = archive.read_file(file_path)
            
            file_name = os.path.basename(file_path)
            output_path = os.path.join(output_dir, file_name)
            with open(output_path, 'wb') as f:
                f.write(file_data)

print("✅ Extracted all .vtf files.")

✅ Extracted all .vtf files.


In [8]:
import os
import subprocess

vtf_folder = r"ExtractedVPK\\"
png_output = r"PNG"
vtfcmd_path = r"vtflib132-bin/bin/x64/VTFCmd.exe" 

# Make sure the output folder exists
os.makedirs(png_output, exist_ok=True)

# Loop through and convert
for filename in os.listdir(vtf_folder):
    if filename.lower().endswith(".vtf"):
        input_path = os.path.join(vtf_folder, filename)
        print("🔄 Converting:", input_path)
        subprocess.run([
            vtfcmd_path,
            "-file", input_path,
            "-output", png_output,
            "-exportformat", "png"
        ])


🔄 Converting: ExtractedVPK\\donev2.vtf


In [18]:
pip install vtf2img

  Obtaining dependency information for vtf2img from https://files.pythonhosted.org/packages/78/04/8248534c3b357e0aaceae5b7728ff49e2244bfd8858962cc53ad77716433/vtf2img-0.1.0-py3-none-any.whl.metadata
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [10]:
import subprocess
import re
import json
import os

vtfcmd_path = r"vtflib132-bin/bin/x64/VTFCmd.exe"
vtf_folder = r"ExtractedVPK\\"
json_output = r"Metadata"
png_output = "PNG"


patterns = {
    "version": r"Version:\s*v(\d+\.\d+)",
    "width": r"Width:\s*(\d+)",
    "height": r"Height:\s*(\d+)",
    "depth": r"Depth:\s*(\d+)",
    "frames": r"Frames:\s*(\d+)",
    "start_frame": r"Start Frame:\s*(\d+)",
    "faces": r"Faces:\s*(\d+)",
    "mipmaps": r"Mipmaps:\s*(\d+)",
    "flags": r"Flags:\s*(0x[0-9A-Fa-f]+)",
    "format": r"Format:\s*([^\n\r]+)",
    "reflectivity": r"Reflectivity:\s*([\d\.]+),\s*([\d\.]+),\s*([\d\.]+)"
}

for filename in os.listdir(vtf_folder):
    vtf_file = os.path.join(vtf_folder, filename)
    result = subprocess.run(
        [vtfcmd_path, "-file", vtf_file , "-output", png_output ,"-exportformat","png"],
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE,
        text=True
    )

    output = result.stdout
    print(output) 

    metadata = {}

    for key, pattern in patterns.items():
        match = re.search(pattern, output)
        if match:
            if key == "reflectivity":
                metadata[key] = {
                    "r": float(match.group(1)),
                    "g": float(match.group(2)),
                    "b": float(match.group(3))
                }
            elif key in ["width", "height", "depth", "frames", "start_frame", "faces", "mipmaps"]:
                metadata[key] = int(match.group(1))
            elif key == "version":
                metadata[key] = float(match.group(1))
            else:
                metadata[key] = match.group(1).strip()

    json_file_path = os.path.join(json_output, filename + ".json")
    
    with open(json_file_path, "w") as f:
        json.dump(metadata, f, indent=4)

    print("Metadata saved to", json_output)


Processing ExtractedVPK\\donev2.vtf...
 Information:
  Version: v7.5
  Size On Disk: 10922.94 KB
  Width: 2048
  Height: 2048
  Depth: 1
  Frames: 2
  Start Frame: 0
  Faces: 1
  Mipmaps: 12
  Flags: 0x00002000
  Bumpmap Scale: 1.00
  Reflectivity: 0.06, 0.01, 0.00
  Format: DXT5

  Resources: 2
 Creating texture:
  Writing PNG\donev2.png...
 Error creating png file.

0/1 files completed.

Metadata saved to Metadata
Processing ExtractedVPK\\mr_04.vtf...
 Information:
  Version: v7.5
  Size On Disk: 4096.30 KB
  Width: 1024
  Height: 1024
  Depth: 1
  Frames: 1
  Start Frame: 65535
  Faces: 6
  Mipmaps: 11
  Flags: 0x00004000
  Bumpmap Scale: 1.00
  Reflectivity: 0.03, 0.03, 0.10
  Format: DXT1

  Resources: 2
 Creating texture:
  Writing PNG\mr_04.png...
 Error creating png file.

0/1 files completed.

Metadata saved to Metadata
Processing ExtractedVPK\\retrosun.vtf...
 Information:
  Version: v7.5
  Size On Disk: 4096.30 KB
  Width: 1024
  Height: 1024
  Depth: 1
  Frames: 1
  Start F

1024 1024


Possible header at offset 0x6b30:
04 af 98 7a ae 26 61 cd 05 cf 44 68 55 e1 bb 1f 7f cf 0f ff 65 ef 63 ed 71 ef 04 21 5c 40 dc dc bc 45 12 35 01 02 b6 58 b5 08 42 61 af a2 0e 3f f4 01 00 f3 28 00 30 ad 63 ef d5 23 af 65 87 80
------------------------------------------------------------
Possible header at offset 0x6b40:
7f cf 0f ff 65 ef 63 ed 71 ef 04 21 5c 40 dc dc bc 45 12 35 01 02 b6 58 b5 08 42 61 af a2 0e 3f f4 01 00 f3 28 00 30 ad 63 ef d5 23 af 65 87 80 ce 83 69 4a aa aa 00 e4 9f 00 98 ba 0f 84 6d 6b
------------------------------------------------------------
Possible header at offset 0x6b50:
bc 45 12 35 01 02 b6 58 b5 08 42 61 af a2 0e 3f f4 01 00 f3 28 00 30 ad 63 ef d5 23 af 65 87 80 ce 83 69 4a aa aa 00 e4 9f 00 98 ba 0f 84 6d 6b 2b ab 04 02 b5 0f 03 cc ed f3 9c 0f 10 84 b5 f5
------------------------------------------------------------
Possible header at offset 0x6b60:
f4 01 00 f3 28 00 30 ad 63 ef d5 23 af 65 87 80 ce 83 69 4a aa aa 00 e4 9f 00 98 ba 0f 84 6d 6b 2b ab 04 